In [1]:
import pandas as pd
from utils.utils import *
from sklearn.linear_model import LinearRegression
import numpy as np

## Model calibration

Read the SMI option prices data

In [2]:
data = pd.read_csv('data.csv', delimiter=';')

Read the SMI option prices data and find :
* the risk free rate $r$ 
* continuously compounded dividend yield $\delta$

In [3]:
# parameters
k = data['Strike']
call = data['Call']
put = data['Put']
S0 = 11118
T=1

#linear regression data
y = call - put - S0
x = np.array(k).reshape((-1, 1))

In [4]:
#linear regression
model = LinearRegression().fit(x, y)

We see if our model fits well the data

In [5]:
#results
print('R^2 : ' + str(model.score(x, y)))
print('intercept:', model.intercept_)
print('slope:', model.coef_[0])

tot_div = -model.intercept_
minus_disc_factor = model.coef_[0]

R^2 : 0.9999999999925049
intercept: -304.8050147058839
slope: -1.0078322426470585


For the rate r : $\beta = -e^{-rT}$ with $T=1\ (years)$ so $r = -\frac{1}{T}ln(-\beta)$ 

For the dividend yield we have: $(1-e^{-\hat{\delta}T})S_0 = \mathcal{D}_0(T)$ so $\hat{\delta} = -\frac{1}{T} ln(1-\frac{D_0(T)}{S_0})$

In [6]:
delta = round(-np.log(1 - tot_div/S0)/T, 6)
print('Dividend yield : ' + str(delta))
r =  round(-np.log(-minus_disc_factor)/T,6)
print('Risk free rate : ' + str(r))


Dividend yield : 0.027798
Risk free rate : -0.007802


We now wish to compute the up and down factors $U$ and $D$ (where $U * D = 1$) using the relation :
$q = \frac{e^{(r - \delta)\Delta} - D }{U - D} \overset{!}{=} 0.5$ in the EEM.

Since the equation won't have real solutions, we use grid search to find the value of $U$ that fits our data the best. 

In [8]:
Delta = 1/12

UP = grid_search(call.values, put.values, r, Delta, delta)
DOWN = round(1 / UP, 4)
print('UP =' + str(UP))
print('DOWN =' + str(DOWN))

UP =1.0491
DOWN =0.9532
